In [1]:
#CSV Bibliothek importieren 
import csv
#Pandas importieren und als pd bezeichnen
import pandas as pd
#Numpy importieren und als np bezeichnen 
import numpy as np
#Json Biliothek importieren
import json
#Obere Schranke für die Größe der Daten auf 100,000,000 setzen  
csv.field_size_limit(100000000)
#Maximale Anzahl von Zeilen und Spalten Zeigen 
pd.options.display.max_columns = None
pd.options.display.max_rows = None
#Extraktion der Domäne
import tldextract

***

# Wissenschaftliche Polarisierung


1. Die Liste der polarisierten Domäne werden vorbereitet.
2. Die Anzahl der benutzten polarisierten Domäne in den Tweets wird berechnet
3. Die Polarisierung für die Benutzer wird berechnet
4. Zum Schluss werden die Ergebnisse gespeichert.

***

### Zuerst werden die Tweets in TweetsCov19 bearbeitet, dann die Tweets in TweetsKB
#### Vorbereitung der Domainsliste

In [2]:
#Hier werden zuerst die Domain-Labels gelesen
pfad_tsv2 = 'C:/Mo/tweetDB/data/labels.csv'

df_label = pd.read_csv(pfad_tsv2)


In [3]:
#Die Quellen und Kategorien, die durch die Quelle unterstützt werden.
pd.options.display.max_columns = None
df_label.head()

,Unnamed: 0,"NewsGuard, Does not repeatedly publish false content","NewsGuard, Gathers and presents information responsibly","NewsGuard, Regularly corrects or clarifies errors","NewsGuard, Handles the difference between news and opinion responsibly","NewsGuard, Avoids deceptive headlines","NewsGuard, Website discloses ownership and financing","NewsGuard, Clearly labels advertising","NewsGuard, Reveals who's in charge, including any possible conflicts of interest","NewsGuard, Provides information about content creators","NewsGuard, score","NewsGuard, overall_class","Pew Research Center, known_by_40%","Pew Research Center, total","Pew Research Center, consistently_liberal","Pew Research Center, mostly_liberal","Pew Research Center, mixed","Pew Research Center, mostly conservative","Pew Research Center, consistently conservative","Wikipedia, is_fake","Open Sources, reliable","Open Sources, fake","Open Sources, unreliable","Open Sources, bias","Open Sources, conspiracy","Open Sources, hate","Open Sources, junksci","Open Sources, rumor","Open Sources, blog","Open Sources, clickbait","Open Sources, political","Open Sources, satire","Open Sources, state","Media Bias / Fact Check, label","Media Bias / Fact Check, factual_reporting","Media Bias / Fact Check, extreme_left","Media Bias / Fact Check, right","Media Bias / Fact Check, extreme_right","Media Bias / Fact Check, propaganda","Media Bias / Fact Check, fake_news","Media Bias / Fact Check, some_fake_news","Media Bias / Fact Check, failed_fact_checks","Media Bias / Fact Check, conspiracy","Media Bias / Fact Check, pseudoscience","Media Bias / Fact Check, hate_group","Media Bias / Fact Check, anti_islam","Media Bias / Fact Check, nationalism","Allsides, bias_rating","Allsides, community_agree","Allsides, community_disagree","Allsides, community_label","BuzzFeed, leaning","PolitiFact, Pants on Fire!","PolitiFact, False","PolitiFact, Mostly False","PolitiFact, Half-True","PolitiFact, Mostly True","PolitiFact, True"
0,21stCenturyWire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conspiracy_pseudoscience,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
1,ABC News,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,95.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_center_bias,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lean Left,8964.0,6949.0,somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AMERICAblog News,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
3,Activist Post,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conspiracy_pseudoscience,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
4,Addicting Info,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,left_bias,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Die Spalte 0 wird zu Domain umbenannt.
df_label.rename(columns = {'Unnamed: 0' : 'domain'}, inplace = True)
df_label['domain'] = df_label['domain'].str.replace(" ", "")
df_label['domain'] = df_label['domain'].str.lower()

In [5]:
pd.options.display.max_columns = None
df_label.head()

,domain,"NewsGuard, Does not repeatedly publish false content","NewsGuard, Gathers and presents information responsibly","NewsGuard, Regularly corrects or clarifies errors","NewsGuard, Handles the difference between news and opinion responsibly","NewsGuard, Avoids deceptive headlines","NewsGuard, Website discloses ownership and financing","NewsGuard, Clearly labels advertising","NewsGuard, Reveals who's in charge, including any possible conflicts of interest","NewsGuard, Provides information about content creators","NewsGuard, score","NewsGuard, overall_class","Pew Research Center, known_by_40%","Pew Research Center, total","Pew Research Center, consistently_liberal","Pew Research Center, mostly_liberal","Pew Research Center, mixed","Pew Research Center, mostly conservative","Pew Research Center, consistently conservative","Wikipedia, is_fake","Open Sources, reliable","Open Sources, fake","Open Sources, unreliable","Open Sources, bias","Open Sources, conspiracy","Open Sources, hate","Open Sources, junksci","Open Sources, rumor","Open Sources, blog","Open Sources, clickbait","Open Sources, political","Open Sources, satire","Open Sources, state","Media Bias / Fact Check, label","Media Bias / Fact Check, factual_reporting","Media Bias / Fact Check, extreme_left","Media Bias / Fact Check, right","Media Bias / Fact Check, extreme_right","Media Bias / Fact Check, propaganda","Media Bias / Fact Check, fake_news","Media Bias / Fact Check, some_fake_news","Media Bias / Fact Check, failed_fact_checks","Media Bias / Fact Check, conspiracy","Media Bias / Fact Check, pseudoscience","Media Bias / Fact Check, hate_group","Media Bias / Fact Check, anti_islam","Media Bias / Fact Check, nationalism","Allsides, bias_rating","Allsides, community_agree","Allsides, community_disagree","Allsides, community_label","BuzzFeed, leaning","PolitiFact, Pants on Fire!","PolitiFact, False","PolitiFact, Mostly False","PolitiFact, Half-True","PolitiFact, Mostly True","PolitiFact, True"
0,21stcenturywire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conspiracy_pseudoscience,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
1,abcnews,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,95.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_center_bias,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lean Left,8964.0,6949.0,somewhat agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,americablognews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
3,activistpost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,conspiracy_pseudoscience,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN
4,addictinginfo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,left_bias,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,left,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#Die beinhalteten Values für Open Sources,conspiracy zählen 
df_label['Open Sources, conspiracy'].value_counts()

1.0    11
2.0     3
3.0     2
Name: Open Sources, conspiracy, dtype: int64

In [9]:
#Die beinhalteten Values für Open Sources,unreliable zählen 
df_label['Open Sources, unreliable'].value_counts()

2.0    9
1.0    4
3.0    2
Name: Open Sources, unreliable, dtype: int64

In [10]:
#Die beinhalteten Values für Open Sources,rumor zählen 
df_label['Open Sources, rumor'].value_counts()

2.0    1
1.0    1
Name: Open Sources, rumor, dtype: int64

In [11]:
#Die beinhalteten Values für Media Bias,conspiracy zählen 
df_label['Media Bias / Fact Check, conspiracy'].value_counts()

0.0    126
1.0      9
Name: Media Bias / Fact Check, conspiracy, dtype: int64

In [12]:
#Die beinhalteten Values für Media Bias,pseudoscience zählen 
df_label['Media Bias / Fact Check, pseudoscience'].value_counts()

0.0    134
1.0      1
Name: Media Bias / Fact Check, pseudoscience, dtype: int64

In [13]:
#Die beinhalteten Values für Media Bias,failed_fact_ckecks zählen 
df_label['Media Bias / Fact Check, failed_fact_checks'].value_counts()

0.0    134
1.0      1
Name: Media Bias / Fact Check, failed_fact_checks, dtype: int64

In [15]:
#Die beinhalteten Values für Media Bias,some_fake_news zählen 
df_label['Media Bias / Fact Check, some_fake_news'].value_counts()

0.0    131
1.0      4
Name: Media Bias / Fact Check, some_fake_news, dtype: int64

In [56]:
#Die beinhalteten Values für Media Bias,junksci zählen 
df_label['Open Sources, junksci'].value_counts()

1.0    2
2.0    1
Name: Open Sources, junksci, dtype: int64

In [17]:
#Die beinhalteten Values für Open Sources,bias zählen 
df_label['Open Sources, bias'].value_counts()

1.0    14
2.0     7
3.0     4
Name: Open Sources, bias, dtype: int64

In [71]:
#Die beinhalteten Values für Open Sources,relible zählen 
df_label['Open Sources, reliable'].value_counts()

2.0    3
Name: Open Sources, reliable, dtype: int64

In [73]:
#Die beinhalteten Values für PolitiFact,True zählen 
df_label['PolitiFact, True'].value_counts()

0.0    21
1.0     2
Name: PolitiFact, True, dtype: int64

In [74]:
#Die beinhalteten Values für PolitiFact,Mostly True zählen 
df_label['PolitiFact, Mostly True'].value_counts()

0.0    20
1.0     3
Name: PolitiFact, Mostly True, dtype: int64

In [75]:
#Die beinhalteten Values für PolitiFact,Half-True zählen 
df_label['PolitiFact, Half-True'].value_counts()

0.0    19
1.0     4
Name: PolitiFact, Half-True, dtype: int64

In [323]:
#Die beinhalteten Values für PolitiFact,bias zählen 
df_label['Open Sources, bias'].value_counts()

1.0    14
2.0     7
3.0     4
Name: Open Sources, bias, dtype: int64

- Anti-Wissenschaft:

In [535]:
#Die Labels von Open Sources, conspiracy, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Open Sources, conspiracy'] == 1.0]

In [536]:
#Die Labels zu Liste umwandeln und speichern.
list_antisc = []
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [537]:
#Die Labels von Open Sources, conspiracy, die mit 2.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Open Sources, conspiracy'] == 2.0]

In [538]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [539]:
#Die Labels von Open Sources, conspiracy, die mit 3.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Open Sources, conspiracy'] == 3.0]

In [540]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [671]:
#Die Labels von Open Sources, unriliable, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Open Sources, unreliable'] == 1.0]

In [672]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [673]:
#Die Labels von Open Sources, unriliable, die mit 2.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Open Sources, unreliable'] == 2.0]

In [674]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [675]:
#Die Labels von Open Sources, unriliable, die mit 3.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Open Sources, unreliable'] == 3.0]

In [676]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [677]:
#Die Labels von PolitiFact, True, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['PolitiFact, True'] == 1.0]

In [678]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [679]:
#Die Labels von Media Bias / Fact Check, some_fake_news, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, some_fake_news'] == 1.0]

In [680]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [695]:
#Die Labels von PolitiFact, Half-True, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['PolitiFact, Half-True'] == 1.0]

In [696]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [697]:
#Die Labels von Media Bias / Fact Check, failed_fact_checks, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, failed_fact_checks'] == 1.0]

In [698]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [681]:
#Die Labels von Open Sources, bias, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Open Sources, bias'] == 1.0]

In [682]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [683]:
#Die Labels von Open Sources, bias, die mit 2.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Open Sources, bias'] == 2.0]

In [684]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [685]:
#Die Labels von Open Sources, bias, die mit 3.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Open Sources, bias'] == 3.0]

In [686]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [543]:
#Die Labels von Open Sources, rumor, die mit 3.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Open Sources, rumor'] == 2.0]

In [544]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [545]:
#Die Labels von Media Bias / Fact Check, conspiracy, die mit 3.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, conspiracy'] == 1.0]

In [546]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [547]:
#Die Labels von Media Bias / Fact Check, pseudoscience, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Media Bias / Fact Check, pseudoscience'] == 1.0]

In [548]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [549]:
#Die Labels von Open Sources, junksci, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Open Sources, junksci'] == 1.0]

In [550]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [551]:
#Die Labels von Open Sources, junksci, die mit 2.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Open Sources, junksci'] == 2.0]

In [552]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_antisc.extend(l_Media_Bias)

In [14]:
list_antisc = np.array(list_antisc)
list_antisc = np.unique(list_antisc)
list_antisc = list_antisc.tolist()

In [15]:
#Anzahl der Domäne, die Anti-Wissenschaftliche Meinungen vertretten.
len(list_antisc)

120

In [11]:
#Liste der Domäne, die Anti-Wissenschaftliche Meinungen vertretten.
list_antisc = np.array(list_antisc)
list_antisc.reshape(1,-1)

array([['21stcenturywire', '369news', '911truth', 'AskingAngels',
        'CampusInsanity', 'ChlorellaFactor', 'CounterThink',
        'abovetopsecret', 'acallforanuprising', 'activistpost',
        'addictinginfo', 'additudemag', 'ae911truth', 'ageofautism',
        'ahtribune', 'aim4truth', 'allianceforadvancedhealth',
        'alternativenews', 'althealthworks', 'americablognews',
        'americandigitalnews', 'americanfreepress', 'americanpolicy',
        'americasfrontlinedoctors', 'amtvmedia', 'ancient-code',
        'ancient-origins', 'anh-usa', 'answersingenesis', 'anyavien',
        'asia-assassinationscience', 'asia-pacificresearch',
        'australiannationalreview', 'autismspeaks', 'awarenessact',
        'beholdisrael', 'biggovernment', 'biodefense', 'biologos',
        'bipartisanreport', 'blacklistednews', 'brandnewtube',
        'breitbart', 'brighteon', 'brightside', 'carm', 'catholic',
        'cbn', 'cfact', 'charismanews', 'childrenshealthdefense',
        'christ

In [ ]:
#Liste der Domäne speichern.
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/dom_antiscience.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for i  in range (0, len(list_antisc)):
        l = []
        l.append(list_antisc[i])
        tsv_writer.writerow(l)

- Pro-Wissenschaft:

In [80]:
list_prosc = []

In [171]:
#Die Labels von PolitiFact, Half-True, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['PolitiFact, Half-True'] == 1.0]

In [172]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_prosc.extend(l_Media_Bias)

In [173]:
#Die Labels von PolitiFact, Mostly True, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['PolitiFact, Mostly True'] == 1.0]

In [174]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_prosc.extend(l_Media_Bias)

In [175]:
#Die Labels von PolitiFact, True, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['PolitiFact, True'] == 1.0]

In [176]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_prosc.extend(l_Media_Bias)

In [177]:
#Die Labels von Open Sources, reliable, die mit 1.0 markiert sind, auswählen.
l_Media_Bias = df_label[df_label['Open Sources, reliable'] == 1.0]

In [178]:
#Die Labels zu Liste umwandeln und speichern.
l_Media_Bias = l_Media_Bias['domain'].tolist()
list_prosc.extend(l_Media_Bias)

In [11]:
list_prosc = np.array(list_prosc)
list_prosc = np.unique(list_prosc)
list_prosc = list_prosc.tolist()

In [12]:
#Anzahl der Domäne, die Pro-Wissenschaftliche Meinungen vertretten.
len(list_prosc)

120

In [10]:
#Liste der Domäne, die Pro-Wissenschaftliche Meinungen vertretten.
list_prosc = np.array(list_prosc)
list_prosc.reshape(1,-1)

array([['Drugs', 'Vaxopedia', 'aap', 'adaa', 'agu', 'aip', 'airspacemag',
        'ajaonline', 'ama-assn', 'aps', 'apsa', 'astronomy',
        'autisticadvocacy', 'bio', 'biomedcentral', 'bmj', 'carbonbrief',
        'cdc', 'climateactiontracker', 'cochrane', 'consumerlab',
        'cosmosmagazine', 'disabilityscoop', 'discovermagazine',
        'earthsky', 'edge', 'energy', 'healthdata', 'hopkinsmedicine',
        'infowars', 'iop', 'irena', 'ispp', 'issues', 'lung',
        'medicalxpress', 'medpagetoday', 'medscape', 'mentaldaily',
        'mentaldaily.com)', 'mosaicscience', 'nasa',
        'nationalacademyofsciences', 'nationalgeographic',
        'nationalreview', 'nature', 'nautil', 'ncbi', 'ncdc', 'nejm',
        'neurosciencenews', 'newatlas', 'newhumanist', 'newscientist',
        'newsweek', 'nextbigfuture', 'nextobserver', 'nih', 'noaa',
        'oceana', 'paho', 'phys', 'plos', 'pnas', 'popsci',
        'popularmechanics', 'psychcentral', 'psychologytoday', 'psypost',
    

In [ ]:
#Liste der Domäne sprichern.
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/dom_proscience_2.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for i  in range (0, len(list_prosc)):
        l = []
        l.append(list_prosc[i])
        tsv_writer.writerow(l)

***

## Polarisierungen berechnen
### TweetsCov19

---

In [2]:
#Die liste der anti-wissenschaftlichen Domäne lesen.
pfad = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/dom_antiscience_2.tsv'

df_dom_ascience = pd.read_csv(pfad, sep='\t', error_bad_lines=False, names=['dom'])

In [3]:
#Die Spalte der anti-wissenschaftlichen Domäne zu Liste umwandeln.
list_antisc = df_dom_ascience.dom.tolist()

In [4]:
list_antisc = np.array(list_antisc)
list_antisc = np.unique(list_antisc)
list_antisc = list_antisc.tolist()

In [5]:
len(list_antisc)

120

In [6]:
#Die liste der pro-wissenschaftlichen Domäne lesen.
pfad = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/dom_proscience_2.tsv'

df_dom_pscience = pd.read_csv(pfad, sep='\t', error_bad_lines=False, names=['dom'])

In [7]:
#Die Spalte der pro-wissenschaftliche Domöne zu Liste umwandeln.
list_prosc = df_dom_pscience.dom.tolist()

In [8]:
list_prosc = np.array(list_prosc)
list_prosc = np.unique(list_prosc)
list_prosc = list_prosc.tolist()

In [9]:
len(list_prosc)

120

In [12]:
#Alle Tweets als Dataframe lesen.
pfad_tsv1 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/TweetsCov19_sample/url/tweetsCov19_id_is_dom.tsv'

df_tweetscov19 = pd.read_csv(pfad_tsv1, sep='\t', error_bad_lines=False, names=['Tweet_Id', 'Username','is_deleted','Urls_num', 'Url_dom'])
pd.options.display.max_columns = None
print('Shape :', df_tweetscov19.shape)
df_tweetscov19.head()

Shape : (9409841, 5)


,Tweet_Id,Username,is_deleted,Urls_num,Url_dom
0,1178791637348212736,c57d4fb895c5f4a33d2a2b24709f93b5,True,0,0
1,1178791641601232896,627f7d4ef584ba33a676a4ebcb861cdb,False,1,catholicnewsagency.com
2,1178791643509612547,7966d5716c1c4f33591b17755184e3d6,False,0,0
3,1178791644214235137,f17996c036ed4cd981f5448fe00d9ed2,False,0,0
4,1178791645371875329,32a54e5f6d8a67de1bffd2e689594dc8,False,0,0


In [13]:
#Diese Methode bekommt die benutzten Domäne als ein String ein.
#Wandelt sie die String-Daten zu Liste um.
#Die Methode geht die Liste durch, prüft jede Domain. 
#Wenn Domain in anti-wissenschaftliche Domänen wäre, dann bekommt der Tweet dafür eine Wertung von -1.

def get_ascience_pol(domains):
    domains_list = domains.split()
    score = 0
    for i in range(0, len(domains_list)):
        
        d = tldextract.extract(domains_list[i])
        d = d.domain
                   
        
        if d in list_antisc:
            score -=  1
        
    return score

In [14]:
#Für alle Tweets wird die Funktion get_ascience_pol() gerufen.
df_tweetscov19['Url_dom'] = df_tweetscov19['Url_dom'].astype(str)
df_tweetscov19['dom_neg_scien'] = df_tweetscov19.apply(lambda row: get_ascience_pol(row['Url_dom']) if (row['Url_dom'] != '0') else 0, axis=1)

In [15]:
df_tweetscov19.head()

,Tweet_Id,Username,is_deleted,Urls_num,Url_dom,dom_neg_scien
0,1178791637348212736,c57d4fb895c5f4a33d2a2b24709f93b5,True,0,0,0
1,1178791641601232896,627f7d4ef584ba33a676a4ebcb861cdb,False,1,catholicnewsagency.com,0
2,1178791643509612547,7966d5716c1c4f33591b17755184e3d6,False,0,0,0
3,1178791644214235137,f17996c036ed4cd981f5448fe00d9ed2,False,0,0,0
4,1178791645371875329,32a54e5f6d8a67de1bffd2e689594dc8,False,0,0,0


In [16]:
#Anzahl der Tweets die eine Wertung weniger als 0 haben.
df_tweetscov19[df_tweetscov19['dom_neg_scien'] < 0].shape

(90732, 6)

In [17]:
#Diese Methode bekommt benutzten Domäne als ein String.
#Wandelt sie die String-Daten zu Liste um.
#Die Methode geht die Liste durch, prüft jede Domain. 
#Wenn Domain in pro-wissenschaftlichen Domäne wäre, dann bekommt der Tweet dafür eine Wertung von 1.

def get_scienc_pos(domains):
    domains_list = domains.split()
    score = 0
    for i in range(0, len(domains_list)):
        
        d = tldextract.extract(domains_list[i])
        d = d.domain
       
            
        
        if d in list_prosc:
            score +=  1
        
    return score

In [18]:
#Für jeden Tweets wird die anzahl der benutzten pro-wissenschaftlichen Domäne berechnet.
df_tweetscov19['dom_pos_scien'] = df_tweetscov19.apply(lambda row: get_scienc_pos(row['Url_dom']) if (row['Url_dom'] != '0') else 0, axis=1)

In [19]:
df_tweetscov19.head()

,Tweet_Id,Username,is_deleted,Urls_num,Url_dom,dom_neg_scien,dom_pos_scien
0,1178791637348212736,c57d4fb895c5f4a33d2a2b24709f93b5,True,0,0,0,0
1,1178791641601232896,627f7d4ef584ba33a676a4ebcb861cdb,False,1,catholicnewsagency.com,0,0
2,1178791643509612547,7966d5716c1c4f33591b17755184e3d6,False,0,0,0,0
3,1178791644214235137,f17996c036ed4cd981f5448fe00d9ed2,False,0,0,0,0
4,1178791645371875329,32a54e5f6d8a67de1bffd2e689594dc8,False,0,0,0,0


In [20]:
df_tweetscov19[df_tweetscov19['dom_pos_scien'] > 0].shape

(67078, 7)

In [21]:
#Hier werdrn die Benutzer in Datframe gruppiert, und die Features 'Anzahl der Domäne', 'Negative Wertungen' und 'Psoitive Wertungen summiert.' 
#dom_neg_scien = Anti-Wissenschaft
#dom_pos_scien = Pro-Wissenschaft
df_tweetscov19_unique_user = df_tweetscov19.groupby('Username')['Urls_num', 'dom_neg_scien', 'dom_pos_scien'].sum()

<ipython-input-21-ab622b5b2d43>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_tweetscov19_unique_user = df_tweetscov19.groupby('Username')['Urls_num', 'dom_neg_scien', 'dom_pos_scien'].sum()


In [22]:
#Anzahl der einzigartigen Users in Dataframe
df_tweetscov19_unique_user.shape

(4095876, 3)

In [23]:
df_tweetscov19_unique_user[df_tweetscov19_unique_user['dom_pos_scien']>0].head()

,Urls_num,dom_neg_scien,dom_pos_scien
Username,,,
0002eb2aff20e181397afe8cdebdd395,1,0,1
00065a00b971b5e1629d504e0ddf2da0,27,-4,1
0006a238065c3acb128041e666b58f07,13,0,2
00084b153bbf7f4a8c1df3b638e07765,15,-9,4
0008667130fe3776676804ff59e8ce3b,5,0,2


In [24]:
#Anzahl der benutzten polarisierten Domäne wird berechnet.
df_tweetscov19_unique_user['pol_num'] = df_tweetscov19_unique_user.apply(lambda row:  (row['dom_pos_scien'] - row['dom_neg_scien']), axis=1)

In [25]:
df_tweetscov19_unique_user[df_tweetscov19_unique_user['dom_neg_scien']<0].head()

,Urls_num,dom_neg_scien,dom_pos_scien,pol_num
Username,,,,
00065a00b971b5e1629d504e0ddf2da0,27,-4,1,5
000795932bf3bc3ab05b5efc1b07104b,2,-1,0,1
00084b153bbf7f4a8c1df3b638e07765,15,-9,4,13
00098f7374d9ea6b43df2c97931ecf01,1,-1,1,2
000df0c86153a1e9e65feca1051f8c1e,7,-4,1,5


In [26]:
#Die Polarisierung der Domäne wird berechnet.
#Anzahl der Negativen + der Positiven durch die Anzahl der benutzten Domäne 
df_tweetscov19_unique_user['user_pol'] = df_tweetscov19_unique_user.apply(lambda row: (row['dom_neg_scien'] + row['dom_pos_scien']) / row['pol_num'] if row['pol_num'] != 0 else 0, axis=1)

In [27]:
df_tweetscov19_unique_user[df_tweetscov19_unique_user['dom_neg_scien']<-2].head()

,Urls_num,dom_neg_scien,dom_pos_scien,pol_num,user_pol
Username,,,,,
00065a00b971b5e1629d504e0ddf2da0,27,-4,1,5,-0.600000
00084b153bbf7f4a8c1df3b638e07765,15,-9,4,13,-0.384615
000df0c86153a1e9e65feca1051f8c1e,7,-4,1,5,-0.600000
0013aaa4da0461c58d0b1306af75af02,215,-14,9,23,-0.217391
0023797612d9ad76ffd7024c324c499c,8,-8,5,13,-0.230769


In [28]:
#Die Benutzer, die eine Wertung von 0 bekommen haben, werden ausgenommen.
df_tweetscov19_unique_user_0 = df_tweetscov19_unique_user[df_tweetscov19_unique_user['user_pol'] != 0]

In [29]:
#Die Form der Dataframe
df_tweetscov19_unique_user_0.shape

(36104, 5)

In [30]:
#Die Form der Dataframe für die Benutzer mit positiven polarisierten Wert.
df_tweetscov19_unique_user_0[df_tweetscov19_unique_user_0['user_pol'] > 0].shape

(19226, 5)

In [31]:
#Die Form der Dataframe für die Benutzer mit negativen polarisierten Wert.
df_tweetscov19_unique_user_0[df_tweetscov19_unique_user_0['user_pol'] < 0].shape

(16878, 5)

In [37]:
#Die polarisieten Werte für Tweets werden gespeichert.
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/tweetscov19_dom_science.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetscov19.iterrows():
        tsv_writer.writerow(row.values.tolist())

In [39]:
df_tweetscov19_unique_user_0 = df_tweetscov19_unique_user_0.reset_index()
df_tweetscov19_unique_user_0.head()

,Username,Urls_num,dom_neg_scien,dom_pos_scien,pol_num,user_pol
0,0002eb2aff20e181397afe8cdebdd395,1,0,1,1,1.000000
1,00065a00b971b5e1629d504e0ddf2da0,27,-4,1,5,-0.600000
2,0006a238065c3acb128041e666b58f07,13,0,2,2,1.000000
3,000795932bf3bc3ab05b5efc1b07104b,2,-1,0,1,-1.000000
4,00084b153bbf7f4a8c1df3b638e07765,15,-9,4,13,-0.384615


In [40]:
#Die Benutzer mit bekannten polarisierten Ansichten speichern. 
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/users/tweetscov19_unique_user_dom_science.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetscov19_unique_user_0.iterrows():
        tsv_writer.writerow(row.values.tolist())

***

## Polarisierungen berechnen
### TweetsKB

***

In [41]:
#Dataframe wird gelesen.
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/tweetskb_sample/url/tweetskb_sample_id_is_urls.tsv'

df_tweetskb = pd.read_csv(pfad_tsv3, sep='\t', error_bad_lines=False, names=['Tweet_Id', "Username", 'is_deleted', 'URLs_dom', 'URLs_num'])
pd.options.display.max_columns = None
print('Shape :', df_tweetskb.shape)
df_tweetskb.head()

Shape : (9409841, 5)


,Tweet_Id,Username,is_deleted,URLs_dom,URLs_num
0,1178791636039589889,DCComics,False,dcuniverseinfinite.com,1
1,1178791641265639424,seanberdyz,True,carrd.co,1
2,1178791640573579264,AnArtistAtBirth,True,0,0
3,1178791644797255680,lagosboygang,True,0,0
4,1178791648056217600,Phenom_Hoops,False,phenomhoopreport.com,1


In [42]:
#Für alle Domäne in Tweets wird die Anzahl der anti-Wissenschaftlichen Domäne berechnet.
df_tweetskb['URLs_dom'] = df_tweetskb['URLs_dom'].astype(str)
df_tweetskb['dom_neg_scien'] = df_tweetskb.apply(lambda row: get_ascience_pol(row['URLs_dom']) if (row['URLs_dom'] != '0') else 0, axis=1)

In [43]:
#Form der Dataframe
df_tweetskb[df_tweetskb['dom_neg_scien'] < 0].shape

(21796, 6)

In [44]:
#Für alle Domäne in Tweets wird die Anzahl der pro-Wissenschaftlichen Domäne berechnet.
df_tweetskb['dom_pos_scien'] = df_tweetskb.apply(lambda row: get_scienc_pos(row['URLs_dom']) if (row['URLs_dom'] != '0') else 0, axis=1)

In [45]:
#Form der Dataframe
df_tweetskb[df_tweetskb['dom_pos_scien'] > 0].shape

(14453, 7)

In [46]:
#Hier werden die Benutzer in Dataframe gruppiert, und die Features 'Anzahl der Domäne', 'Negative Wertungen' und 'Psoitive Wertungen summiert.' 
#dom_neg_scien = Anti-Wissenschaft
#dom_pos_scien = Pro-Wissenschaft
df_tweetskb_unique_user = df_tweetskb.groupby('Username')['URLs_num', 'dom_neg_scien', 'dom_pos_scien'].sum()

<ipython-input-46-185a630db4bb>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_tweetskb_unique_user = df_tweetskb.groupby('Username')['URLs_num', 'dom_neg_scien', 'dom_pos_scien'].sum()


In [47]:
df_tweetskb_unique_user.shape

(5208425, 3)

In [48]:
df_tweetskb_unique_user.head()

,URLs_num,dom_neg_scien,dom_pos_scien
Username,,,
00000000,0,0,0
000000000yt,0,0,0
00000000martin,0,0,0
000000911a,0,0,0
000000_127,0,0,0


In [49]:
#Die Anzahl der benutzten polarisierten Domäne wird berechnet.
df_tweetskb_unique_user['pol_num'] = df_tweetskb_unique_user.apply(lambda row: (row['dom_pos_scien'] - row['dom_neg_scien']), axis=1)

In [50]:
df_tweetskb_unique_user[df_tweetskb_unique_user['dom_neg_scien']<-2].head()

,URLs_num,dom_neg_scien,dom_pos_scien,pol_num
Username,,,,
0nehunter69,4,-3,3,6
10thAmendment,10,-8,7,15
123RHGreen,54,-18,17,35
1776AmericaUSA,12,-7,4,11
1776Resistance,13,-3,0,3


In [51]:
#Die Polarisierungswerte für die Benutzer wird berechnet.
df_tweetskb_unique_user['user_pol'] = df_tweetskb_unique_user.apply(lambda row: (row['dom_neg_scien'] + row['dom_pos_scien']) / row['pol_num'] if row['pol_num'] != 0 else 0, axis=1)

In [52]:
#Die Benutzer, die eine Wertung von 0 bekommen haben, werden ausgenommen.
df_tweetskb_unique_user_0 = df_tweetskb_unique_user[df_tweetskb_unique_user['user_pol'] != 0]

In [53]:
#Die Form der Dataframe 
df_tweetskb_unique_user_0.shape

(11311, 5)

In [54]:
#Die Benutzer, die eine positive Wertung haben. 
df_tweetskb_unique_user_0[df_tweetskb_unique_user_0['user_pol'] > 0].shape

(5019, 5)

In [55]:
#Die Benutzer, die eine negative Wertung haben. 
df_tweetskb_unique_user_0[df_tweetskb_unique_user_0['user_pol'] < 0].shape

(6292, 5)

In [71]:
#Die polarisierten Werte für die Tweets werden gespeichert.
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/tweetskb_dom_schience.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetskb.iterrows():
        tsv_writer.writerow(row.values.tolist())

In [69]:
df_tweetskb_unique_user_0 = df_tweetskb_unique_user_0.reset_index()
df_tweetskb_unique_user_0.head()

,Username,URLs_num,dom_neg_scien,dom_pos_scien,pol_num,user_pol
0,01_zebra,4,-1,0,1,-1.0
1,01sth02,1,0,1,1,1.0
2,02MK13B,1,-1,0,1,-1.0
3,08_diane08,1,-1,0,1,-1.0
4,0Ludicris5,4,-2,0,2,-1.0


In [70]:
df_tweetskb_unique_user_0.shape

(11311, 6)

In [72]:
#Die Benutzer mit polarisierten Werte werden gespeichert.
pfad_tsv3 = 'C:/Mo/tweetDB/tweetscov19/TweetsCov19_tsv/data_sample/dom_label/users/tweetskb_unique_user_dom_science.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetskb_unique_user_0.iterrows():
        tsv_writer.writerow(row.values.tolist())